# Anatomy of a Transaction
This notebook contains scripts to create a database and table, insert data to the table, and return records from the transaction log showing the activity for each step.

The script uses function ```fn_dblog``` to return transaction log records.
```fn_dblog``` is a table-valued function that takes two parameters, begin and end LSN.  If NULL is passed for both parameters all active rows in the log will be returned.
The following statement will return all active records from the transaction log for the master database.  The result set contains 127 columns.


In [ ]:
USE master;

SELECT * FROM fn_dblog(NULL,NULL);

Because ```fn_dblog``` is a table-valued function, you can use a column list and WHERE predicate.  The following query will return all active rows from the master database transaction log, but a subset of columns.

In [ ]:
USE master;

SELECT [Current LSN]
,[Transaction ID]
,[Transaction Name]
,Operation
,Context
,[Description]
,[Previous LSN]
,AllocUnitName
,[Page ID]
,[Slot ID]
,[Begin Time]
,[Database Name]
,[Number of Locks]
,[Lock Information]
,[New Split Page]
FROM fn_dblog(NULL,NULL);

THe Transaction Name column is only populated at the beginning of a transaction.  If the transaction is explicitly named, that name will appear here.  If the transaction is not named, SQL Server will give it a name.

In [ ]:
SELECT [Current LSN]
,[Transaction ID]
,[Transaction Name]
,Operation
,Context
,[Description]
,[Previous LSN]
,AllocUnitName
,[Page ID]
,[Slot ID]
,[Begin Time]
,[Database Name]
,[Number of Locks]
,[Lock Information]
,[New Split Page]
FROM fn_dblog(NULL,NULL)
WHERE [Transaction Name] IS NOT NULL;

The Operation column contains the action SQL Server is taking in this log record.

In [ ]:
SELECT DISTINCT Operation
FROM fn_dblog(NULL,NULL)
ORDER BY Operation;

The Context column contains the object type being operated on.

In [ ]:
SELECT DISTINCT Context
FROM fn_dblog(NULL,NULL)
ORDER BY Context;

The Description column describes the action taken by the log record.  Be warned, it is not always terribly descriptive.

In [ ]:
SELECT [Current LSN]
,[Transaction ID]
,[Transaction Name]
,Operation
,Context
,[Description]
,[Previous LSN]
,AllocUnitName
,[Page ID]
,[Slot ID]
,[Begin Time]
,[Database Name]
,[Number of Locks]
,[Lock Information]
,[New Split Page]
FROM fn_dblog(NULL,NULL);

Each log record contains an identifier called the log sequence number (LSN).  Because SQL Server is not a single-threaded process, log records from different transactions will be interleaved in the log file.  SQL Server uses the previous_lsn_number to maintain order.

In [ ]:
SELECT [Transaction ID]
,Operation
,[Current LSN]
,[Previous LSN]
FROM fn_dblog(NULL,NULL)
WHERE [Transaction ID] <> '0000:00000000';

In [ ]:
SELECT [Current LSN]
,[Transaction ID]
,[Transaction Name]
,Operation
,Context
,[Description]
,[Previous LSN]
,AllocUnitName
,[Page ID]
,[Slot ID]
,[Begin Time]
,[Database Name]
,[Number of Locks]
,[Lock Information]
,[New Split Page]
FROM fn_dblog(NULL,NULL)
WHERE AllocUnitName = 'dbo.CommandLog.PK_CommandLog';

Finally, each log record contains the following columns:
* AllocUnitName - the object being affected
* Page ID - the ID of the data page being affected
* Slot ID - the ID of the slot containing the row being affected (NOTE - Slot ID is 0 based, so the first row on the page will be in slot 0)
* Number of Locks - The number of locks being held by this log record
* Lock Information - Contains information about the locks being held (Generally in format OBJECT:PAGE:ROW)

In [3]:
SELECT [Current LSN]
,[Transaction ID]
,[Transaction Name]
,Operation
,Context
,[Description]
,[Previous LSN]
,AllocUnitName
,[Page ID]
,[Slot ID]
,[Begin Time]
,[Database Name]
,[Number of Locks]
,[Lock Information]
,[New Split Page]
FROM fn_dblog(NULL,NULL)
WHERE AllocUnitName = 'dbo.CommandLog.PK_CommandLog';

(11 rows affected)

Total execution time: 00:00:00.031

Current LSN,Transaction ID,Transaction Name,Operation,Context,Description,Previous LSN,AllocUnitName,Page ID,Slot ID,Begin Time,Database Name,Number of Locks,Lock Information,New Split Page
000000e7:00000038:0003,0000:00002b09,NULL,LOP_MODIFY_COLUMNS,LCX_CLUSTERED,,000000e7:00000038:0001,dbo.CommandLog.PK_CommandLog,0001:0000020e,0,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (8194443284a0),NULL
000000e7:00000040:0004,0000:00002b0a,NULL,LOP_INSERT_ROWS,LCX_CLUSTERED,,000000e7:00000040:0003,dbo.CommandLog.PK_CommandLog,0001:0000020e,1,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (61a06abd401c),NULL
000000e7:00000048:0002,0000:00002b0b,NULL,LOP_MODIFY_COLUMNS,LCX_CLUSTERED,,000000e7:00000048:0001,dbo.CommandLog.PK_CommandLog,0001:0000020e,1,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (61a06abd401c),NULL
000000e7:00000050:0002,0000:00002b0c,NULL,LOP_INSERT_ROWS,LCX_CLUSTERED,,000000e7:00000050:0001,dbo.CommandLog.PK_CommandLog,0001:0000020e,2,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (98ec012aa510),NULL
000000e7:00000068:0003,0000:00002b0f,NULL,LOP_MODIFY_COLUMNS,LCX_CLUSTERED,,000000e7:00000068:0002,dbo.CommandLog.PK_CommandLog,0001:0000020e,2,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (98ec012aa510),NULL
000000e7:00000070:0002,0000:00002b10,NULL,LOP_INSERT_ROWS,LCX_CLUSTERED,,000000e7:00000070:0001,dbo.CommandLog.PK_CommandLog,0001:0000020e,3,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (a0c936a3c965),NULL
000000e7:00000078:0002,0000:00002b11,NULL,LOP_MODIFY_COLUMNS,LCX_CLUSTERED,,000000e7:00000078:0001,dbo.CommandLog.PK_CommandLog,0001:0000020e,3,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (a0c936a3c965),NULL
000000e7:00000080:0002,0000:00002b12,NULL,LOP_INSERT_ROWS,LCX_CLUSTERED,,000000e7:00000080:0001,dbo.CommandLog.PK_CommandLog,0001:0000020e,4,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (59855d342c69),NULL
000000e7:00000098:0003,0000:00002b15,NULL,LOP_MODIFY_COLUMNS,LCX_CLUSTERED,,000000e7:00000098:0002,dbo.CommandLog.PK_CommandLog,0001:0000020e,4,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (59855d342c69),NULL
000000e7:000000a0:0002,0000:00002b16,NULL,LOP_INSERT_ROWS,LCX_CLUSTERED,,000000e7:000000a0:0001,dbo.CommandLog.PK_CommandLog,0001:0000020e,5,NULL,NULL,3,HoBt 72057594043498496:ACQUIRE_LOCK_IX OBJECT: 1:279672044:0 ;ACQUIRE_LOCK_IX PAGE: 1:1:526 ;ACQUIRE_LOCK_X KEY: 1:72057594043498496 (b9b173bbe8d5),NULL
